<a href="https://colab.research.google.com/github/tadie1921/RL_stock_prediction/blob/main/reinforcement_stock_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing all the necessary liberaries

In [ ]:

!pip install yfinance
!pip install ta
!pip install -U gym==0.21.0
!pip install -U quanttrader==0.5.5
!pip install -U pyfolio==0.9.2

!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet
!pip install -U PyYaml==3.13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=a9871f51a6e3b073d41b473f758589db48c90eaa3039a3368a11dc4902b5e5c8
  Stored in directory: /root/.cache/pip/wheels/6a/76/03/7f785aaa50b9c6ec7e3fd105a62c1b2c45a034512d51e024a9
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× E

In [ ]:
import os
import io
import tempfile
import shutil
import zipfile
from google.colab import files

from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
import gym
import quanttrader as qt
from quanttrader import TradingEnv
import pyfolio as pf

import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.environments import tf_py_environment
from tf_agents.environments import suite_gym
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential, q_network, network
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

/usr/local/lib/python3.9/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
gym.__version__, qt.__version__, pf.__version__

('0.23.0', '0.5.5', '0.9.2')

# Data collection and preprocessing

---



---



In [ ]:
from datetime import datetime, timedelta
import ta
def load_data():
    start_date = datetime(2010, 1, 1)
    end_date = datetime(2020, 12, 31)
    syms = ['SPY']
    max_price_scaler = 5_000.0
    max_volume_scaler = 1.5e8
    df_obs = pd.DataFrame()             # observation
    df_exch = pd.DataFrame()            # exchange; for order match

    for sym in syms:
        df = yf.download(sym, start=start_date, end=end_date)
        df.index = pd.to_datetime(df.index) + timedelta(hours=15, minutes=59, seconds=59)

        df_exch = pd.concat([df_exch, df['Close'].rename(sym)], axis=1)

        df['Open'] = df['Adj Close'] / df['Close'] * df['Open'] / max_price_scaler
        df['High'] = df['Adj Close'] / df['Close'] * df['High'] / max_price_scaler
        df['Low'] = df['Adj Close'] / df['Close'] * df['Low'] / max_price_scaler
        df['Volume'] = df['Adj Close'] / df['Close'] * df['Volume'] / max_volume_scaler
        df['Close'] = df['Adj Close'] / max_price_scaler
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        df.columns = [f'{sym}:{c.lower()}' for c in df.columns]

        macd = ta.trend.MACD(close=df[f'{sym}:close'])
        df[f'{sym}:macd'] = macd.macd()
        df[f'{sym}:macd_diff'] = macd.macd_diff()
        df[f'{sym}:macd_signal'] = macd.macd_signal()

        rsi = ta.momentum.RSIIndicator(close=df[f'{sym}:close'])
        df[f'{sym}:rsi'] = rsi.rsi()

        bb = ta.volatility.BollingerBands(close=df[f'{sym}:close'], window=20, window_dev=2)
        df[f'{sym}:bb_bbm'] = bb.bollinger_mavg()
        df[f'{sym}:bb_bbh'] = bb.bollinger_hband()
        df[f'{sym}:bb_bbl'] = bb.bollinger_lband()

        atr = ta.volatility.AverageTrueRange(high=df[f'{sym}:high'], low=df[f'{sym}:low'], close=df[f'{sym}:close'])
        df[f'{sym}:atr'] = atr.average_true_range()

        df_obs = pd.concat([df_obs, df], axis=1)

    return df_obs, df_exch

# Load the data
df_obs, df_exch = load_data()

# View the data
display(df_obs.tail(15))
print(df_exch)

[*********************100%***********************]  1 of 1 completed


,SPY:open,SPY:high,SPY:low,SPY:close,SPY:volume,SPY:macd,SPY:macd_diff,SPY:macd_signal,SPY:rsi,SPY:bb_bbm,SPY:bb_bbh,SPY:bb_bbl,SPY:atr
Date,,,,,,,,,,,,,
2020-12-09 15:59:59,0.071453,0.071486,0.070503,0.070677,0.475856,0.001121,0.000016,0.001106,61.770441,0.069791,0.071728,0.067854,0.000856
2020-12-10 15:59:59,0.070392,0.070871,0.070211,0.070654,0.370774,0.001062,-0.000035,0.001097,61.548651,0.069888,0.071794,0.067983,0.000843
2020-12-11 15:59:59,0.070301,0.070656,0.069985,0.070571,0.370538,0.000997,-0.000080,0.001077,60.707321,0.070014,0.071742,0.068287,0.000830
2020-12-14 15:59:59,0.071022,0.071245,0.070218,0.070255,0.444503,0.000909,-0.000134,0.001043,57.480169,0.070078,0.071742,0.068413,0.000844
2020-12-15 15:59:59,0.070783,0.071205,0.070498,0.071205,0.410140,0.000906,-0.000110,0.001016,63.723207,0.070145,0.071876,0.068414,0.000852
2020-12-16 15:59:59,0.071249,0.071507,0.071066,0.071316,0.375174,0.000902,-0.000091,0.000993,64.385773,0.070237,0.072011,0.068463,0.000822
2020-12-17 15:59:59,0.071657,0.071758,0.071486,0.071715,0.411772,0.000920,-0.000058,0.000978,66.721637,0.070391,0.072117,0.068665,0.000795
2020-12-18 15:59:59,0.071775,0.071810,0.071011,0.071429,0.880606,0.000901,-0.000062,0.000963,63.501046,0.070516,0.072161,0.068871,0.000796
2020-12-21 15:59:59,0.070614,0.073224,0.070046,0.071173,0.621629,0.000856,-0.000086,0.000941,60.687375,0.070652,0.072019,0.069284,0.000966


                            SPY
2010-01-04 15:59:59  113.330002
2010-01-05 15:59:59  113.629997
2010-01-06 15:59:59  113.709999
2010-01-07 15:59:59  114.190002
2010-01-08 15:59:59  114.570000
...                         ...
2020-12-23 15:59:59  367.570007
2020-12-24 15:59:59  369.000000
2020-12-28 15:59:59  372.170013
2020-12-29 15:59:59  371.459991
2020-12-30 15:59:59  371.989990

[2768 rows x 1 columns]


# Building Trading Environment

In [ ]:
# Set parameters for the trading environment
window_size = 5
cash = 100_000.0
max_nav_scaler = cash

# Create training environment
train_qt_env = TradingEnv(2, df_obs, df_exch)
train_qt_env.set_cash(cash)
train_qt_env.set_commission(0.0001)
train_qt_env.set_steps(n_lookback=window_size, n_warmup=50, n_maxsteps=250)
train_qt_env.set_feature_scaling(max_nav_scaler)

# Create evaluation environment
eval_qt_env = TradingEnv(2, df_obs, df_exch)
eval_qt_env.set_cash(cash)
eval_qt_env.set_commission(0.0001)
eval_qt_env.set_steps(n_lookback=window_size, n_warmup=50, n_maxsteps=2000, n_init_step=504)
eval_qt_env.set_feature_scaling(max_nav_scaler)

# Reset the evaluation environment and print the observation for the first 3 time steps
o1 = eval_qt_env.reset()
idx0 = eval_qt_env._init_step
idx1 = idx0 + 3
print(eval_qt_env._df_obs_scaled[idx0:idx1])

total_reward = 0.0
while True:
    # Take a random action (for demonstration purposes only)
    action = 1
    o2, reward, done, info = eval_qt_env.step(action)
    total_reward += reward

    # Print the action, reward, and info (for demonstration purposes only)
    print(action, reward * max_nav_scaler, info)

    if done:
        break

# Print the shapes of the initial and final observations
print(o1.shape, o2.shape)




                     SPY:open  SPY:high   SPY:low  SPY:close  SPY:volume  \
Date                                                                       
2012-01-03 15:59:59  0.020683  0.020783  0.020629   0.020640    1.045233   
2012-01-04 15:59:59  0.020592  0.020691  0.020513   0.020673    0.686324   
2012-01-05 15:59:59  0.020561  0.020759  0.020467   0.020728    0.938372   

                     SPY:macd  SPY:macd_diff  SPY:macd_signal    SPY:rsi  \
Date                                                                       
2012-01-03 15:59:59  0.000174       0.000056         0.000118  60.324434   
2012-01-04 15:59:59  0.000195       0.000062         0.000133  60.780481   
2012-01-05 15:59:59  0.000213       0.000064         0.000149  61.588820   

                     SPY:bb_bbm  SPY:bb_bbh  SPY:bb_bbl   SPY:atr  
Date                                                               
2012-01-03 15:59:59    0.020118    0.020768    0.019467  0.000328  
2012-01-04 15:59:59    0.020136   

The observation has 10 features, and lookback is 14 days.

In [ ]:
idx0 = eval_qt_env._init_step
idx1 = idx0+3
eval_qt_env._df_obs_scaled[idx0:idx1]         # observation

,SPY:open,SPY:high,SPY:low,SPY:close,SPY:volume,SPY:macd,SPY:macd_diff,SPY:macd_signal,SPY:rsi,SPY:bb_bbm,SPY:bb_bbh,SPY:bb_bbl,SPY:atr
Date,,,,,,,,,,,,,
2012-01-03 15:59:59,0.020683,0.020783,0.020629,0.020640,1.045233,0.000174,0.000056,0.000118,60.324477,0.020118,0.020768,0.019467,0.000328
2012-01-04 15:59:59,0.020592,0.020691,0.020513,0.020673,0.686324,0.000195,0.000062,0.000133,60.780547,0.020136,0.020826,0.019446,0.000318
2012-01-05 15:59:59,0.020561,0.020759,0.020467,0.020728,0.938372,0.000213,0.000064,0.000149,61.588802,0.020157,0.020891,0.019423,0.000316


In [ ]:
eval_qt_env._df_exch[idx0:idx1]

,SPY
2012-01-03 15:59:59,127.500000
2012-01-04 15:59:59,127.699997
2012-01-05 15:59:59,128.039993


At the end of 2012-01-03, if action = 1 or we all in SPY, then we buy 100_000/127.50 or 784 shares, commission=784x127.50x0.0001=9.996, and the remaining cash=100_000-784x127.5-9.996=30.

Then the market moves to 2012-01-04, and SPY price goes up to 127.70. Our 784 shares are now worth 784x127.70, and NAV including cash becomes 784x127.70+30=100_146.80.

NAV change is the reward, in this case is 146.80.

As shown below.

In [ ]:
eval_qt_env._df_positions.iloc[idx0]

SPY          0.0
Cash    100000.0
NAV     100000.0
Name: 2012-01-03 15:59:59, dtype: float64

In [ ]:
eval_qt_env._df_positions.iloc[idx0+1]

SPY        784.000000
Cash        30.004000
NAV     100146.801607
Name: 2012-01-04 15:59:59, dtype: float64

# Create TF-Agents environment from Gym environment.

In [ ]:
train_qt_env = gym.wrappers.FlattenObservation(train_qt_env)
train_py_env = suite_gym.wrap_env(train_qt_env)
train_env = tf_py_environment.TFPyEnvironment(train_py_env)

eval_qt_env = gym.wrappers.FlattenObservation(eval_qt_env)
eval_py_env = suite_gym.wrap_env(eval_qt_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [ ]:
train_env.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(1))

In [ ]:
train_env.time_step_spec()

TimeStep(
{'discount': BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)),
 'observation': BoundedTensorSpec(shape=(140,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32)),
 'reward': TensorSpec(shape=(), dtype=tf.float32, name='reward'),
 'step_type': TensorSpec(shape=(), dtype=tf.int32, name='step_type')})

# Some helper functions for visualization

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

def create_policy_eval_video(env, policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = env.reset()
      video.append_data(env.pyenv.envs[0].render())

      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = env.step(action_step.action)
        video.append_data(env.pyenv.envs[0].render())

  return embed_mp4(filename)

# Spontaneous Trader

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())
time_step = train_py_env.reset()
random_policy.action_spec

BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(1))

In [ ]:
time_step = train_env.reset()
action_step = random_policy.action(time_step)

Below figure shows spontaneous trader's random trading behavior.The upper half is SPY price curve along with red buy and green sell marks. The lower half is NAV or total asset value.

In [ ]:
create_policy_eval_video(train_env, random_policy, "random-agent", num_episodes=1)

In [ ]:
train_env.pyenv.envs[0].env._df_positions

,SPY,Cash,NAV
2010-01-04 15:59:59,0.0,100000.0,100000.0
2010-01-05 15:59:59,0.0,100000.0,100000.0
2010-01-06 15:59:59,0.0,100000.0,100000.0
2010-01-07 15:59:59,0.0,100000.0,100000.0
2010-01-08 15:59:59,0.0,100000.0,100000.0
...,...,...,...
2020-12-23 15:59:59,0.0,0.0,0.0
2020-12-24 15:59:59,0.0,0.0,0.0
2020-12-28 15:59:59,0.0,0.0,0.0
2020-12-29 15:59:59,0.0,0.0,0.0


## Reinforcement Trader

In [ ]:
learning_rate = 1e-3
num_eval_episodes = 10
replay_buffer_max_length = 100000

In [ ]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

Here we use a three layer fully-connected deep neural network. The input observation is 14 days x 10 features, flattened to shape 140. The first layer has 100 neurons. Therefore, it requires $140 \times 100+100=14,100$ parameters.

The second layer has 50 neurons, implying $100 \times 50 + 50 = 5,050$ parameters. The output layer is a binary decision of either buy or sell, which needs $50 \times 2 + 2 = 102$ parameters.

In [ ]:
q_net.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  14100     
                                                                 
 dense_1 (Dense)             multiple                  5050      
                                                                 
 dense_2 (Dense)             multiple                  102       
                                                                 
Total params: 19,252
Trainable params: 19,252
Non-trainable params: 0
_________________________________________________________________


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())

In [ ]:
time_step = train_env.reset()

In [ ]:
random_policy.action(time_step)

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>, state=(), info=())

In [ ]:
def compute_avg_return(environment, policy, num_episodes=5):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0
    zeros = 0
    ones = 0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      if action_step.action.numpy()[0] == 1:
        ones+=1
      else:
        zeros+=1
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0], zeros, ones

In [ ]:
compute_avg_return(eval_env, random_policy, num_episodes=1)

(41200.676, 995, 1005)

In [ ]:
replay_buffer = TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=100_000)

In [ ]:
replay_buffer_observer = replay_buffer.add_batch

In [ ]:
train_env.reset()

init_driver = DynamicStepDriver(
    train_env,
    random_policy,
    observers=[replay_buffer.add_batch],
    num_steps=2_500)
final_time_step, final_policy_state = init_driver.run()

In [ ]:
trajectories, buffer_info = replay_buffer.get_next(sample_batch_size=2, num_steps=3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [ ]:
trajectories.observation.shape

TensorShape([2, 3, 140])

In [ ]:
from tf_agents.trajectories.trajectory import to_transition
time_steps, action_steps, next_time_steps = to_transition(trajectories)
time_steps.observation.shape

TensorShape([2, 2, 140])

Each row of the replay buffer only stores a single observation step. But since the DQN Agent needs both the current and next observation to compute the loss, the dataset pipeline will sample two adjacent rows for each item in the batch (`num_steps=2`).

In [ ]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=64,
    num_steps=2).prefetch(3)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


In [ ]:
dataset

In [ ]:
iterator = iter(dataset)

In [ ]:
num_iterations = 1_000_000   # less intelligence, more persistance; 24x7 player
save_interval = 100_000
eval_interval = 50_000
log_interval = 5_000

In [ ]:
# Create a driver to collect experience.
collect_driver = DynamicStepDriver(
    train_env,
    agent.collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=4) # collect 4 steps for each training iteration

### Training the Trader

In [ ]:
# (Optional) Optimize by wrapping some of the code in a graph using TF function.
collect_driver.run = common.function(collect_driver.run)
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(train_env, agent.policy, num_episodes=1)[0]
returns = np.array([avg_return])

# Reset the environment.
time_step = None
policy_state = agent.collect_policy.get_initial_state(train_env.batch_size)

In [ ]:
num_iterations = 20000
while True:
    # Collect a few steps using collect_policy and save to the replay buffer.
    time_step, policy_state = collect_driver.run(time_step, policy_state)

    # Sample a batch of data from the buffer and update the agent's network.
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()
    print(f'\r step {step}', end='')

    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss))

    if step % eval_interval == 0:
        avg_return = compute_avg_return(train_env, agent.policy, num_episodes=1)[0]
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns = np.append(returns, avg_return)

    # if step % save_interval == 0:
    #     save_checkpoint_to_local()

    if step > num_iterations:
        break

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


 step 5000step = 5000: loss = 1551350.375
 step 10000step = 10000: loss = 1306787.5
 step 15000step = 15000: loss = 890927.125
 step 20000step = 20000: loss = 875180.0
 step 20001

### Evaluate Trader Performance

In [ ]:
create_policy_eval_video(eval_env, agent.policy, "trained-agent", num_episodes=1)

In [ ]:
compute_avg_return(eval_env, random_policy, num_episodes=1)

x_left = eval_env.pyenv.envs[0].env._init_step
x_right = eval_env.pyenv.envs[0].env._current_step     # _maxsteps+1
df_price = eval_env.pyenv.envs[0].env._df_exch[x_left:x_right].copy()
df_spontaneous = eval_env.pyenv.envs[0].env._df_positions['NAV'][x_left:x_right].copy()

In [ ]:
print(compute_avg_return(eval_env, saved_policy, num_episodes=1))
df_reinforcement = eval_env.pyenv.envs[0].env._df_positions['NAV'][x_left:x_right].copy()

(139012.14, 438, 1562)


In [ ]:
df_all = pd.concat([df_spontaneous, df_reinforcement, df_price], axis=1)
df_all.columns = ['spontaneous', 'tf-agent', 'benchmark']
df_ret = df_all / df_all.shift(1) - 1
df_ret = df_ret[1:]
random_perf_stats = pf.timeseries.perf_stats(df_ret['spontaneous'])
agent_perf_stats = pf.timeseries.perf_stats(df_ret['tf-agent'])
benchmark_perf_stats = pf.timeseries.perf_stats(df_ret['benchmark'])
perf_stats = pd.concat([random_perf_stats, agent_perf_stats, benchmark_perf_stats], axis=1)
perf_stats.columns = ['random', 'tf-agent', 'benchmark']
perf_stats

,random,tf-agent,benchmark
Annual return,0.079287,0.116104,0.121725
Cumulative returns,0.831727,1.390122,1.487294
Annual volatility,0.090710,0.113069,0.128985
Sharpe ratio,0.886646,1.028216,0.955339
Calmar ratio,0.551160,0.776651,0.603244
Stability,0.962235,0.940382,0.950932
Max drawdown,-0.143855,-0.149494,-0.201785
Omega ratio,1.247272,1.230952,1.185798
Sortino ratio,1.283426,1.496901,1.345309
Skew,-0.251531,-0.130487,-0.401799


The reinforcement trader outperforms random spontaneous trader. the TF agent delivers slightly less annual return than buy-and-hold (11.6% vs 12.17%), but with better risk-adjusted measures (Sharpe 1.028 vs 0.955).